<a href="https://colab.research.google.com/github/shahchhatru/AI_colab_notebooks/blob/main/LaMini_FLan_t5_24_news_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [3]:
from transformers import pipeline

checkpoint = "MBZUAI/LaMini-Flan-T5-248M"

model = pipeline('text2text-generation', model = checkpoint)

input_prompt = 'Please let me know your thoughts on the given place and why you think it deserves to be visited: \n"Barcelona, Spain"'
generated_text = model(input_prompt, max_length=512, do_sample=True)[0]['generated_text']

print("Response", generated_text)

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Response As an AI language model, I do not have personal opinions nor thoughts. However, I can provide information, recommendations, and factual information about the world's most popular tourist destinations during Barcelona's time of year and the reasons why they should be visited in the future. Whether you are in Barcelona or not, Barcelona is a city that is often considered one of the most beautiful and iconic places to visit in Europe. The city is known for its cultural diversity, rich history, and stunning architecture, and the city has a rich history that continues to influence countless people around the world.
